# A191 Stock Factors 021-040

In [1]:
import dai
import pandas as pd
from a191_sql_base import get_a191_sql_base

In [2]:
sd = '2023-01-01'
ed = '2026-01-01'

## A191 Factor 021

Formula:

`REGBETA(MEAN(CLOSE,6),SEQUENCE(6))`

In [3]:
sql_alpha_021 = f"""--sql
SELECT
    date,
    instrument,
    m_regr_slope(m_avg(close,6), row_number(6), 6) 
    AS alpha_a191_f0021
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_021), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0021
55750,2023-01-18,000001.SZ,7.791067
55751,2023-01-18,000002.SZ,-10.986670
55752,2023-01-18,000004.SZ,-0.127721
55753,2023-01-18,000005.SZ,-0.017211
55754,2023-01-18,000006.SZ,-2.456900
...,...,...,...
3858340,2025-12-26,688799.SH,0.304137
3858341,2025-12-26,688800.SH,1.117863
3858345,2025-12-26,688819.SH,-0.057436
3858346,2025-12-26,688981.SH,1.488476


## A191 Factor 022

Formula:

`SMEAN(((CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)-DELAY((CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6),3)),12,1)`


In [4]:
sql_alpha_022 = f"""--sql
SELECT
    date,
    instrument,
    m_ta_ewm(((close-m_avg(close,6))/m_avg(close,6)-m_lag((close-m_avg(close,6))/m_avg(close,6),3)),12,1) 
    AS alpha_a191_f0022
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_022), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0022
45612,2023-01-16,000001.SZ,0.013203
45613,2023-01-16,000002.SZ,-0.001805
45614,2023-01-16,000004.SZ,0.042839
45615,2023-01-16,000005.SZ,0.013319
45616,2023-01-16,000006.SZ,-0.008696
...,...,...,...
3858340,2025-12-26,688799.SH,-0.000952
3858341,2025-12-26,688800.SH,0.013274
3858345,2025-12-26,688819.SH,0.003757
3858346,2025-12-26,688981.SH,0.006599


## A191 Factor 023

Formula:

`SMA((CLOSE>DELAY(CLOSE,1)?STD(CLOSE:20),0),20,1)/(SMA((CLOSE>DELAY(CLOSE,1)?STD(CLOSE,20):0),20,1)+SMA((CLOSE<=DELAY(CLOSE,1)?STD(CLOSE,20):0),20,1))*100`


In [5]:
sql_alpha_023 = f"""--sql
SELECT
    date,
    instrument,
    (m_ta_ewm(IF(close > m_lag(close, 1), m_stddev(close, 20), 0), 20, 1) /
     (m_ta_ewm(IF(close > m_lag(close, 1), m_stddev(close, 20), 0), 20, 1) +
      m_ta_ewm(IF(close <= m_lag(close, 1), m_stddev(close, 20), 0), 20, 1))) * 100 
    AS alpha_a191_f0023
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_023), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0023
101438,2023-02-07,000001.SZ,100.000000
101439,2023-02-07,000002.SZ,100.000000
101440,2023-02-07,000004.SZ,100.000000
101441,2023-02-07,000005.SZ,0.000000
101442,2023-02-07,000006.SZ,100.000000
...,...,...,...
3858340,2025-12-26,688799.SH,51.463248
3858341,2025-12-26,688800.SH,50.680675
3858345,2025-12-26,688819.SH,47.012673
3858346,2025-12-26,688981.SH,42.184843


## A191 Factor 024

Formula:

`SMA(CLOSE-DELAY(CLOSE,5),5,1)`


In [6]:
sql_alpha_024 = f"""--sql
SELECT
    date,
    instrument,
    m_ta_ewm(close-m_lag(close,5),5,1) 
    AS alpha_a191_f0024
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_024), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0024
30405,2023-01-11,000001.SZ,39.877674
30406,2023-01-11,000002.SZ,-13.825922
30407,2023-01-11,000004.SZ,-2.600872
30408,2023-01-11,000005.SZ,-0.370704
30409,2023-01-11,000006.SZ,-19.080379
...,...,...,...
3858341,2025-12-26,688800.SH,6.492309
3858342,2025-12-26,688802.SH,-134.460000
3858345,2025-12-26,688819.SH,0.335658
3858346,2025-12-26,688981.SH,5.655768


## A191 Factor 025

Formula:

`((-1 * RANK((DELTA(CLOSE, 7) * (1 -RANK(DECAYLINEAR((VOLUME/MEAN(VOLUME,20)), 9)))))) * (1 +RANK(SUM(RET, 250))))`


In [7]:
sql_alpha_025 = f"""--sql
SELECT
    date,
    instrument,
    ((-1 * c_pct_rank((m_delta(close, 7) * (1 -c_pct_rank(m_decay_linear((volume/m_avg(volume,20)), 9)))))) *
     (1 +c_pct_rank(m_sum(ret, 250)))) 
    AS alpha_a191_f0025
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_025), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0025
1303681,2024-01-12,000001.SZ,-0.602959
1303682,2024-01-12,000002.SZ,-0.006507
1303684,2024-01-12,000005.SZ,-0.441990
1303685,2024-01-12,000006.SZ,-0.399510
1303687,2024-01-12,000008.SZ,-0.649524
...,...,...,...
3858340,2025-12-26,688799.SH,-0.692092
3858341,2025-12-26,688800.SH,-1.840275
3858345,2025-12-26,688819.SH,-0.726377
3858346,2025-12-26,688981.SH,-1.531414


## A191 Factor 026

Formula:

`((((SUM(CLOSE, 7) / 7) -CLOSE)) + ((CORR(VWAP, DELAY(CLOSE, 5), 230))))`


In [8]:
sql_alpha_026 = f"""--sql
SELECT
    date,
    instrument,
    (((m_sum(close, 7) / 7) -close)) + ((m_corr(vwap, m_lag(close, 5), 230))) 
    AS alpha_a191_f0026
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_026), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0026
1223663,2023-12-21,000001.SZ,-3.034508
1223664,2023-12-21,000002.SZ,12.385963
1223666,2023-12-21,000005.SZ,1.294678
1223667,2023-12-21,000006.SZ,5.890152
1223669,2023-12-21,000008.SZ,1.776122
...,...,...,...
3858340,2025-12-26,688799.SH,1.958132
3858341,2025-12-26,688800.SH,-9.507441
3858345,2025-12-26,688819.SH,-0.054785
3858346,2025-12-26,688981.SH,-1.826923


## A191 Factor 027

Formula:

`WMA((CLOSE-DELAY(CLOSE,3))/DELAY(CLOSE,3)*100+(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100,12)`


In [9]:
sql_alpha_027 = f"""--sql
SELECT
    date,
    instrument,
    m_ta_wma((close-m_lag(close,3))/m_lag(close,3)*100+
             (close-m_lag(close,6))/m_lag(close,6)*100,12) 
    AS alpha_a191_f0027
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_027), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0027
91279,2023-02-03,000001.SZ,-1.482372
91280,2023-02-03,000002.SZ,-3.354742
91281,2023-02-03,000004.SZ,7.650748
91282,2023-02-03,000005.SZ,-1.679079
91283,2023-02-03,000006.SZ,0.475630
...,...,...,...
3858340,2025-12-26,688799.SH,1.893226
3858341,2025-12-26,688800.SH,3.475683
3858345,2025-12-26,688819.SH,1.969120
3858346,2025-12-26,688981.SH,7.160394


## A191 Factor 028

Formula:

`3*SMA((CLOSE-TSMIN(LOW,9))/(TSMAX(HIGH,9)-TSMIN(LOW,9))*100,3,1)-2*SMA(SMA((CLOSE-TSMIN(LOW,9))/(MAX(HIGH,9)-TSMAX(LOW,9))*100,3,1),3,1)`


In [10]:
sql_alpha_028 = f"""--sql
SELECT
    date,
    instrument,
    3*m_ta_ewm((close-m_min(low,9))/(m_max(high,9)-m_min(low,9))*100,3,1) -
    2*m_ta_ewm(m_ta_ewm((close-m_min(low,9))/(m_max(high,9)-m_max(low,9))*100,3,1),3,1) 
    AS alpha_a191_f0028
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_028), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0028
45612,2023-01-16,000001.SZ,-410.782241
45613,2023-01-16,000002.SZ,-65.942029
45614,2023-01-16,000004.SZ,-202.315789
45615,2023-01-16,000005.SZ,-50.769231
45616,2023-01-16,000006.SZ,-4.041812
...,...,...,...
3858340,2025-12-26,688799.SH,-84.443544
3858341,2025-12-26,688800.SH,-95.929347
3858345,2025-12-26,688819.SH,-79.916161
3858346,2025-12-26,688981.SH,-305.480170


## A191 Factor 029

Formula:

`(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*VOLUME`


In [11]:
sql_alpha_029 = f"""--sql
SELECT
    date,
    instrument,
    (close-m_lag(close,6))/m_lag(close,6)*volume 
    AS alpha_a191_f0029
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_029), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0029
35474,2023-01-12,000001.SZ,1.529282e+06
35475,2023-01-12,000002.SZ,-1.674484e+06
35476,2023-01-12,000004.SZ,-4.625853e+04
35477,2023-01-12,000005.SZ,1.861926e+05
35478,2023-01-12,000006.SZ,-4.677483e+06
...,...,...,...
3858341,2025-12-26,688800.SH,1.630900e+06
3858342,2025-12-26,688802.SH,-5.919999e+05
3858345,2025-12-26,688819.SH,1.247876e+05
3858346,2025-12-26,688981.SH,3.317643e+06


## A191 Factor 030

Formula:

`WMA((REGRESI(CLOSE/DELAY(CLOSE)-1,MKT,SMB,HML，60))^2,20)`


In [12]:
sql_alpha_030 = f"""--sql
SELECT
    date,
    instrument,
    m_ta_wma((m_ols3d_last_resid(close/m_lag(close,1)-1,MKT,SMB,HML,60))^2,20) 
    AS alpha_a191_f0030
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_030), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0030
408819,2023-05-08,000001.SZ,0.000088
408820,2023-05-08,000002.SZ,0.000293
408821,2023-05-08,000004.SZ,0.000758
408822,2023-05-08,000005.SZ,0.000303
408823,2023-05-08,000006.SZ,0.000117
...,...,...,...
3858340,2025-12-26,688799.SH,0.000290
3858341,2025-12-26,688800.SH,0.001571
3858345,2025-12-26,688819.SH,0.000225
3858346,2025-12-26,688981.SH,0.000225


## A191 Factor 031

Formula:

`(CLOSE-MEAN(CLOSE,12))/MEAN(CLOSE,12)*100`


In [13]:
sql_alpha_031 = f"""--sql
SELECT
    date,
    instrument,
    (close-m_avg(close,12))/m_avg(close,12)*100 
    AS alpha_a191_f0031
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_031), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0031
60821,2023-01-19,000001.SZ,2.189616
60822,2023-01-19,000002.SZ,-1.521855
60823,2023-01-19,000004.SZ,2.127660
60824,2023-01-19,000005.SZ,0.665779
60825,2023-01-19,000006.SZ,-4.527238
...,...,...,...
3858340,2025-12-26,688799.SH,-1.025818
3858341,2025-12-26,688800.SH,6.917228
3858345,2025-12-26,688819.SH,2.334445
3858346,2025-12-26,688981.SH,4.449736


## A191 Factor 032

Formula:

`(-1 * SUM(RANK(CORR(RANK(HIGH), RANK(VOLUME), 3)), 3))`


In [14]:
sql_alpha_032 = f"""--sql
SELECT
    date,
    instrument,
    (-1 * m_sum(c_pct_rank(m_corr(c_pct_rank(high), c_pct_rank(volume), 3)), 3)) 
    AS alpha_a191_f0032
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_032), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0032
25337,2023-01-10,000001.SZ,-0.929120
25338,2023-01-10,000002.SZ,-1.217936
25339,2023-01-10,000004.SZ,-1.366271
25340,2023-01-10,000005.SZ,-1.937351
25341,2023-01-10,000006.SZ,-2.346829
...,...,...,...
3858342,2025-12-26,688802.SH,-1.179568
3858344,2025-12-26,688807.SH,-1.320467
3858345,2025-12-26,688819.SH,-0.826682
3858346,2025-12-26,688981.SH,-1.009865


## A191 Factor 033

Formula:

`((((-1 * TSMIN(LOW, 5)) + DELAY(TSMIN(LOW, 5), 5)) * RANK(((SUM(RET, 240) -SUM(RET, 20)) / 220))) *TSRANK(VOLUME, 5))`


In [15]:
sql_alpha_033 = f"""--sql
SELECT
    date,
    instrument,
    ((((-1 * m_min(low, 5)) + m_lag(m_min(low, 5), 5)) *
      c_pct_rank(((m_sum(ret, 240) -m_sum(ret, 20)) / 220))) *
     m_rank(volume, 5)) 
    AS alpha_a191_f0033
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_033), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0033
1250315,2023-12-28,000001.SZ,-1.361816
1250316,2023-12-28,000002.SZ,3.922258
1250318,2023-12-28,000005.SZ,0.312311
1250319,2023-12-28,000006.SZ,6.362813
1250321,2023-12-28,000008.SZ,0.637968
...,...,...,...
3858340,2025-12-26,688799.SH,-7.966189
3858341,2025-12-26,688800.SH,-2.257231
3858345,2025-12-26,688819.SH,0.316482
3858346,2025-12-26,688981.SH,-2.813336


## A191 Factor 034

Formula:

`MEAN(CLOSE,12)/CLOSE`


In [16]:
sql_alpha_034 = f"""--sql
SELECT
    date,
    instrument,
    m_avg(close,12)/close 
    AS alpha_a191_f0034
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_034), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0034
60821,2023-01-19,000001.SZ,0.978573
60822,2023-01-19,000002.SZ,1.015454
60823,2023-01-19,000004.SZ,0.979167
60824,2023-01-19,000005.SZ,0.993386
60825,2023-01-19,000006.SZ,1.047419
...,...,...,...
3858340,2025-12-26,688799.SH,1.010364
3858341,2025-12-26,688800.SH,0.935303
3858345,2025-12-26,688819.SH,0.977188
3858346,2025-12-26,688981.SH,0.957398


## A191 Factor 035

Formula:

`(MIN(RANK(DECAYLINEAR(DELTA(OPEN, 1), 15)), RANK(DECAYLINEAR(CORR((VOLUME), ((OPEN * 0.65) +(OPEN *0.35)), 17),7))) * -1)`


In [17]:
sql_alpha_035 = f"""--sql
SELECT
    date,
    instrument,
    (least(c_pct_rank(m_decay_linear(m_delta(open, 1), 15)),
           c_pct_rank(m_decay_linear(m_corr((volume), ((open * 0.65) +(open *0.35)), 17),7))) * -1) 
    AS alpha_a191_f0035
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_035), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0035
76071,2023-01-31,000034.SZ,-0.666667
76076,2023-01-31,000039.SZ,-1.000000
76083,2023-01-31,000050.SZ,-0.333333
81122,2023-02-01,000001.SZ,-0.983607
81123,2023-02-01,000002.SZ,-0.000615
...,...,...,...
3858340,2025-12-26,688799.SH,-0.454298
3858341,2025-12-26,688800.SH,-0.622356
3858345,2025-12-26,688819.SH,-0.093150
3858346,2025-12-26,688981.SH,-0.479138


## A191 Factor 036

Formula:

`RANK(SUM(CORR(RANK(VOLUME), RANK(VWAP)), 6), 2)`


In [18]:
sql_alpha_036 = f"""--sql
SELECT
    date,
    instrument,
    c_pct_rank(m_sum(m_corr(c_pct_rank(volume), c_pct_rank(vwap), 6), 2)) 
    AS alpha_a191_f0036
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_036), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0036
35474,2023-01-12,000001.SZ,0.109975
35475,2023-01-12,000002.SZ,0.353843
35476,2023-01-12,000004.SZ,0.275348
35477,2023-01-12,000005.SZ,0.815822
35478,2023-01-12,000006.SZ,0.959935
...,...,...,...
3858341,2025-12-26,688800.SH,0.781117
3858342,2025-12-26,688802.SH,0.360023
3858345,2025-12-26,688819.SH,0.147538
3858346,2025-12-26,688981.SH,0.732261


## A191 Factor 037

Formula:

`(-1 * RANK(((SUM(OPEN, 5) * SUM(RET, 5)) -DELAY((SUM(OPEN,5) * SUM(RET, 5)), 10))))`


In [19]:
sql_alpha_037 = f"""--sql
SELECT
    date,
    instrument,
    (-1 * c_pct_rank(((m_sum(open, 5) * m_sum(ret, 5)) -
                      m_lag((m_sum(open,5) * m_sum(ret, 5)), 10)))) 
    AS alpha_a191_f0037
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_037), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0037
76045,2023-01-31,000001.SZ,-0.004302
76046,2023-01-31,000002.SZ,-0.001844
76047,2023-01-31,000004.SZ,-0.898381
76048,2023-01-31,000005.SZ,-0.308543
76049,2023-01-31,000006.SZ,-0.990985
...,...,...,...
3858340,2025-12-26,688799.SH,-0.064210
3858341,2025-12-26,688800.SH,-0.061300
3858345,2025-12-26,688819.SH,-0.190689
3858346,2025-12-26,688981.SH,-0.863240


## A191 Factor 038

Formula:

`(((SUM(HIGH, 20) / 20) < HIGH) ? (-1 * DELTA(HIGH, 2)) : 0)`


In [20]:
sql_alpha_038 = f"""--sql
SELECT
    date,
    instrument,
    (IF(((m_sum(high, 20) / 20) < high), (-1 * m_delta(high, 2)), 0)) 
    AS alpha_a191_f0038
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_038), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0038
5067,2023-01-04,000001.SZ,0.000000
5068,2023-01-04,000002.SZ,0.000000
5069,2023-01-04,000004.SZ,0.000000
5070,2023-01-04,000005.SZ,0.000000
5071,2023-01-04,000006.SZ,0.000000
...,...,...,...
3858343,2025-12-26,688805.SH,0.000000
3858344,2025-12-26,688807.SH,0.000000
3858345,2025-12-26,688819.SH,-1.312227
3858346,2025-12-26,688981.SH,-0.410000


## A191 Factor 039

Formula:

`((RANK(DECAYLINEAR(DELTA((CLOSE), 2),8)) -RANK(DECAYLINEAR(CORR(((VWAP * 0.3) + (OPEN * 0.7)),SUM(MEAN(VOLUME,180), 37), 14), 12))) * -1)`


In [21]:
sql_alpha_039 = f"""--sql
SELECT
    date,
    instrument,
    ((c_pct_rank(m_decay_linear(m_delta((close), 2),8)) -
      c_pct_rank(m_decay_linear(m_corr(((vwap * 0.3) + (open * 0.7)),
                                          m_sum(m_avg(volume,180), 37), 14), 12))) * -1) 
    AS alpha_a191_f0039
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_039), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0039
1250315,2023-12-28,000001.SZ,-0.115560
1250316,2023-12-28,000002.SZ,-0.052071
1250318,2023-12-28,000005.SZ,-0.101356
1250319,2023-12-28,000006.SZ,0.382440
1250321,2023-12-28,000008.SZ,0.421731
...,...,...,...
3858340,2025-12-26,688799.SH,0.348854
3858341,2025-12-26,688800.SH,-0.728346
3858345,2025-12-26,688819.SH,-0.216934
3858346,2025-12-26,688981.SH,-0.539962


## A191 Factor 040

Formula:

`SUM((CLOSE>DELAY(CLOSE,1)?VOLUME:0),26)/SUM((CLOSE<=DELAY(CLOSE,1)?VOLUME:0),26)*100`


In [22]:
sql_alpha_040 = f"""--sql
SELECT
    date,
    instrument,
    (m_sum(IF(close > m_lag(close, 1), volume, 0), 26) /
     m_sum(IF(close <= m_lag(close, 1), volume, 0), 26)) * 100 
    AS alpha_a191_f0040
FROM data_base
"""
dai.query(get_a191_sql_base(sql_alpha_040), filters={'date': [sd, ed]}).df().dropna()

,date,instrument,alpha_a191_f0040
131944,2023-02-15,000001.SZ,84.620389
131945,2023-02-15,000002.SZ,66.684795
131946,2023-02-15,000004.SZ,100.361036
131947,2023-02-15,000005.SZ,46.939078
131948,2023-02-15,000006.SZ,49.545993
...,...,...,...
3858340,2025-12-26,688799.SH,150.347182
3858341,2025-12-26,688800.SH,154.386419
3858345,2025-12-26,688819.SH,122.596550
3858346,2025-12-26,688981.SH,87.613904
